# Accelerating lines and curves <img align="right" width="192" height="64" src="https://freemelt.com/app/uploads/freemeltLogo-1.png">
Accelerating lines are useful when working with highly heat conductive materials, such as tungsten. When line melting tungsten with a constant traverse speed, the heat will traverse the powder ahead of the beam in the form of a wave, causing overmelting. Accelerating the lines can allow for "catching up" to the heatwave and mitigate overmelting. 

Accelerating lines are also useful for configuring porosity in a material: a uniformly accelerating line will ensure a smooth porosity gradient in the build object. 

In [1]:
import obplib as obp
import notebook_viewer as nw
import numpy as np
import os

## The `AcceleratingLine`/`AcceleratingCurve` objects
In most respects, the accelerating paths are the same as their nonaccelerating counterparts. The difference is, of course, the acceleration.

The nonaccelerating objects have a `Speed`/`speed` attribute which is constant along the whole path. The accelerating paths instead have `si` (initial velocity) and `sf` (final velocity) attributes. The velocity of the beam will go from `si` to `sf` with a constant acceleration during its traverse. 

### Example - Half Circle
In this example, a half circle is created using accelerating lines. It's a variant of the last example in [the `Curve` guide](länk), with acceleration added.

In [2]:
def accelerating_bezier_half_circle(midpoint=[0, 0], radius=1):
    #Define the coefficients for the bezier curve
    a, b, c = 1.00005519, 0.55342686, 0.99873585

    curves = []
    for i in range(2):
        #Define the control coordinates for the bezier curve
        control_coordinates = [[0, -a], [b, -c], [c, -b], [a, -0]]  # mirroring in the y axis

        #Scale the control coordinates with the radius
        control_coordinates = [np.multiply(control_coordinates[i], radius) for i in range(len(control_coordinates))]

        #Mirroring the control points in the x and y plane to create the four parts
        if i == 1:
            control_coordinates = [[np.multiply(control_coordinates[i][0], -1), control_coordinates[i][1]] for i in range(len(control_coordinates))]

        #Translate the control coordinates with the midpoint
        control_coordinates = [np.add(control_coordinates[i], midpoint) for i in range(len(control_coordinates))]

        #Create Point objects from the control coordinates
        control_points = [obp.Point(control_coordinates[i][0], control_coordinates[i][1]) for i in range(len(control_coordinates))]

        #Create circle segments and add them to the curves list
        curves.append(obp.AcceleratingCurve(control_points[0], control_points[1], control_points[2], control_points[3], si=100, sf=200, bp=obp.Beamparameters(spot_size=100, power=100)))

    #Create line segment and add it to the curves list
    line_points = [obp.Point(-a, 0), obp.Point(a, 0)]
    curves.append(obp.AcceleratingLine(line_points[0], line_points[1], si=200, sf=100, bp=obp.Beamparameters(spot_size=100, power=100)))

    return curves

#Create the curves
cur = accelerating_bezier_half_circle()


filename = 'accelerating_half_circle.obp'
#Remove the file if it already exists
try:
    os.remove(filename)
except:
    pass

#Write the curves to the file
obp.write_obp(cur, filename)

#Display the curves using the interactive viewer
nw.notebook_viewer(cur)


interactive(children=(IntSlider(value=2, description='upper_lim', max=3, min=1), IntText(value=3, description=…